## **Set directories**

In [ ]:
import os

tools_path = "./tools"
assert os.path.exists(tools_path), f"tools path does not exists: {tools_path}"

DATASETS = "./data"
dtd_path = DATASETS + "/dtd"
mvtec_path = DATASETS + "/mvtec"
assert os.path.exists(dtd_path), f"DTD path does not exists: {dtd_path}"
assert os.path.exists(mvtec_path), f"MVTEC path does not exists: {mvtec_path}"

OUTPUT_PATH = "./result"
assert os.path.exists(OUTPUT_PATH), f"DTD path does not exists: {OUTPUT_PATH}"


print("Directories set.")

## **Install the library required**

In [ ]:
%pip install -r {WORKDIR}/requirements.txt

## **Generate foreground and global retrieval result**

In [ ]:
!python {tools_path}/generate_foreground.py \
    -lp {OUTPUT_PATH}/foreground \
    --data_dir {DATASETS} \
    --vis

!python {tools_path}/generate_retrieval.py \
    -lp {OUTPUT_PATH}/retrieval \
    --data_dir {DATASETS} \
    --vis \
    -k 5


## **Train**

### ***Generate synthetic data***

In [ ]:
!python {tools_path}/generate_synthetic_data.py \
    -fd {OUTPUT_PATH}/foreground \
    -lp {OUTPUT_PATH}/synthetic_mvtec

### ***Run***

*Please set the OUTPUT_PATH which is same as before*

In [ ]:
%%bash

# objects=('bottle' 'cable' 'capsule' 'hazelnut' 'metal_nut' 'pill' 'screw' 'toothbrush' 'transistor' 'zipper')
# textures=('carpet' 'grid' 'leather' 'tile' 'wood')
objects=('bottle')
OUTPUT_PATH="./result"

# [10] bottle test result: pro: 0.9593 ap: 0.8097 pixel-auc: 0.9819 image-auc: 1.0000
# [20] bottle test result: pro: 0.9619 ap: 0.8193 pixel-auc: 0.9836 image-auc: 1.0000
# [50] bottle test result: pro: 0.9701 ap: 0.8754 pixel-auc: 0.9872 image-auc: 1.0000

# python train.py -lp log/mvtec_train --sub-categories "${textures[@]}" \
# -fd ./log/foreground_mvtec_DenseNet_features.denseblock1_320/ \
# --steps 500 \
# -tps 50 \
# --data-dir log/synthetic_mvtec_640_12000_True_jpg

python train.py \
    -lp ${OUTPUT_PATH}/mvtec_train \
    --sub-categories "${objects[@]}" \
    -fd ${OUTPUT_PATH}/foreground/ \
    -rd ${OUTPUT_PATH}/retrieval \
    --steps 2 \
    -tps 1 \
    --data-dir ${OUTPUT_PATH}/synthetic_mvtec

## **Test**

In [ ]:
!python test.py \
    -lp result/mvtec_test \
    -fd result/foreground/ \
    -rd result/retrieval \
    --checkpoints weights/{category}.pth

## **Open the mlflow ui**

In [ ]:
import mlflow

!mlflow ui --backend-store-uri file:'./mlruns' 